In [3]:
# Cell 1: Install libraries
!pip install mediapipe opencv-python pandas scikit-learn numpy matplotlib

  Using cached mediapipe-0.10.13-cp311-cp311-macosx_11_0_universal2.whl (50.1 MB)
  Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl (35.4 MB)
  Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
  Using cached scikit_learn-1.4.2-cp311-cp311-macosx_12_0_arm64.whl (10.5 MB)
  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
  Using cached matplotlib-3.8.4-cp311-cp311-macosx_11_0_arm64.whl (7.5 MB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 929.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 1.8 MB/s eta 0:00:0000:0100:01
  Using cached opencv_contrib_python-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl (44.2 MB)
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
  Using cached sounddevi

In [4]:
# Cell 2: Libraries
import mediapipe as mp
import cv2
import numpy as np
import time
import pandas as pd
import csv
import os
from matplotlib import pyplot as plt

In [5]:
# Cell 3: Check that the camera and mediapipe are working
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, \
        mp.solutions.hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                print("Error: Unable to read frame from video capture")
                break

            # Flip image to simulate mirror view
            image = cv2.flip(image, 1)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detections
            pose_results = pose.process(image)
            hand_results = hands.process(image)

            # RGB 2 BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw points
            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
                )

            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp.solutions.hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                        mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                    )

            cv2.imshow("Raw Webcam Feed", image)

            if cv2.waitKey(10) & 0xFF == ord("q"):
                break

    cap.release()
    cv2.destroyAllWindows()


2024-05-08 11:10:41.133 python[5123:138801] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1715159442.393856  138801 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1715159442.399487  138801 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
/Users/jorgemuyo/Desktop/ML_Craftsmanship/.venv/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
# Cell 4: Record video for data capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30 
    print(f"Video Resolution: {width}x{height} at {fps} FPS")

    # Define the codec
    fourcc = cv2.VideoWriter_fourcc(*'XVID') 
    out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Error: Unable to read frame from video capture")
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb.flags.writeable = False
        results = hands.process(frame_rgb)  
        frame_rgb.flags.writeable = True
        frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

        # Write every frame
        out.write(frame)

        cv2.imshow('MediaPipe Hands', frame)  
        if cv2.waitKey(5) & 0xFF == ord('q'): # 'q' to quit
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [6]:
# Cell 5: Create CSV to store data
headers = ['class', 'accuracy', 'sequence'] 
headers.extend([f'pose_{coord}{i}' for i in range(33) for coord in ('x', 'y', 'z', 'v')])
headers.extend([f'{hand}_{coord}{i}' for hand in ('right_hand', 'left_hand') for i in range(21) for coord in ('x', 'y', 'z', 'v')])

with open('coordinates_1.csv', mode='w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(headers)

In [8]:
# Cell 6: Capture data for first 'movement'
cap = cv2.VideoCapture('_.mp4')
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('processed_output.avi', fourcc, fps, (width, height))

    # MediaPipe initialization
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    pose_drawing_spec = mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4)
    hand_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)

    record = False
    accuracy = None
    sequence = -1  
    recording_state = None  

    with open('coordinates_1.csv', mode='a', newline='') as file:
        csv_writer = csv.writer(file)

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Error: Unable to read frame from video capture")
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb.flags.writeable = False
            pose_results = pose.process(frame_rgb)
            hand_results = hands.process(frame_rgb)
            frame_rgb.flags.writeable = True
            frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

            # Drawing the landmarks
            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=pose_drawing_spec,
                    connection_drawing_spec=pose_drawing_spec)

            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        landmark_drawing_spec=hand_drawing_spec,
                        connection_drawing_spec=hand_drawing_spec)

            row = ['', accuracy, sequence if record else -1]

            if pose_results.pose_landmarks:
                for lm in pose_results.pose_landmarks.landmark:
                    visibility_binary = 1 if lm.visibility > 0.3 else 0
                    row.extend([lm.x, lm.y, lm.z, visibility_binary])
            else:
                row.extend([0] * 33 * 4)

            for hand in ('right_hand', 'left_hand'):
                found = False
                if hand_results.multi_hand_landmarks:
                    for hand_landmarks, handedness in zip(hand_results.multi_hand_landmarks, hand_results.multi_handedness):
                        if handedness.classification[0].label == ('Right' if hand == 'right_hand' else 'Left'):
                            for lm in hand_landmarks.landmark:
                                visibility_binary = 1 if lm.visibility > 0.2 else 0
                                row.extend([lm.x, lm.y, lm.z, visibility_binary])
                            found = True
                            break
                if not found:
                    row.extend([0] * 21 * 4)

            if record:
                csv_writer.writerow(row)

            out.write(frame)
            cv2.imshow('MediaPipe Pose', frame)
            key = cv2.waitKey(5) & 0xFF
            if key == ord('r') or key == ord('w'):
                new_state = chr(key)
                if new_state != recording_state:
                    sequence += 1  # Increment sequence only when state changes
                    recording_state = new_state
                record = True
                accuracy = 1 if key == ord('r') else 0
            elif key == ord('s'):
                record = False
                recording_state = None  # Reset recording state
            elif key == ord('q'):
                break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# write 'class' for first movement
df = pd.read_csv('coordinates_1.csv')
df.loc[df['accuracy'].notna(), 'class'] = 'movement_1'  
df.to_csv('coordinates_1.csv', index=False)


I0000 00:00:1715159732.279918  138801 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
I0000 00:00:1715159732.284361  138801 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
/Users/jorgemuyo/Desktop/ML_Craftsmanship/.venv/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/var/folders/wv/_x9hjmys03x5gnbfl70ry2sr0000gn/T/ipykernel_5123/1173199726.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'movement_1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['accuracy'].notna(), 'class'] = 'movement_1'  # Only fill 'class' where there's recorded data


In [12]:
# Cell 7: Capture data for second 'movement'
cap = cv2.VideoCapture('_.mp4')
if not cap.isOpened():
    print("Error: Unable to open video capture")
else:
    # Retrieving video properties
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    # Setting up video writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('processed_output.avi', fourcc, fps, (width, height))

    # MediaPipe pose and hand solutions initialization
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    pose_drawing_spec = mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4)
    hand_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)

    record = False
    accuracy = None
    sequence = -1  
    recording_state = None  

    # Open the CSV file for appending data
    with open('coordinates_2.csv', mode='a', newline='') as file:
        csv_writer = csv.writer(file)

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Error: Unable to read frame from video capture")
                break

            # Convert the frame to RGB for processing
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb.flags.writeable = False
            pose_results = pose.process(frame_rgb)
            hand_results = hands.process(frame_rgb)
            frame_rgb.flags.writeable = True
            frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

            # Drawing the landmarks on the frame
            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=pose_drawing_spec,
                    connection_drawing_spec=pose_drawing_spec)

            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        landmark_drawing_spec=hand_drawing_spec,
                        connection_drawing_spec=hand_drawing_spec)

            # Initialize the row with label 'movement_2'
            row = ['movement_2', accuracy if accuracy is not None else '', sequence if record else -1]

            # Append pose landmarks to the row
            if pose_results.pose_landmarks:
                for lm in pose_results.pose_landmarks.landmark:
                    visibility_binary = 1 if lm.visibility > 0.3 else 0
                    row.extend([lm.x, lm.y, lm.z, visibility_binary])
            else:
                row.extend([0] * 33 * 4)

            # Append hand landmarks to the row
            for hand in ('right_hand', 'left_hand'):
                found = False
                if hand_results.multi_hand_landmarks:
                    for hand_landmarks, handedness in zip(hand_results.multi_hand_landmarks, hand_results.multi_handedness):
                        if handedness.classification[0].label == ('Right' if hand == 'right_hand' else 'Left'):
                            for lm in hand_landmarks.landmark:
                                visibility_binary = 1 if lm.visibility > 0.2 else 0
                                row.extend([lm.x, lm.y, lm.z, visibility_binary])
                            found = True
                            break
                if not found:
                    row.extend([0] * 21 * 4)

            # Write the row to CSV if recording is active
            if record:
                csv_writer.writerow(row)

            # Show the frame
            out.write(frame)
            cv2.imshow('MediaPipe Pose', frame)
            key = cv2.waitKey(5) & 0xFF
            if key == ord('r') or key == ord('w'):
                new_state = chr(key)
                if new_state != recording_state:
                    sequence += 1  
                    recording_state = new_state
                record = True
                accuracy = 1 if key == ord('r') else 0
            elif key == ord('s'):
                record = False
                recording_state = None
            elif key == ord('q'):
                break

    cap.release()
    out.release()
    cv2.destroyAllWindows()


I0000 00:00:1715161177.503264  138801 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
I0000 00:00:1715161177.507710  138801 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro


In [13]:
# Cell 8: Join all the CSVs of the different movements
df1 = pd.read_csv('coordinates_1.csv', header=None)
df2 = pd.read_csv('coordinates_2.csv', header=None)

# Concatenate along rows
combined_df = pd.concat([df1, df2], axis=0, ignore_index=True)

# Save combined DataFrame to a new CSV
combined_path = 'combined_coordinates.csv'
combined_df.to_csv(combined_path, header=False, index=False)

In [14]:
# Cell 9: Delete not important data
data = pd.read_csv('combined_coordinates.csv')

# Display column names to understand what columns are actually in the dataset
print(data.columns.tolist())

# Define columns to remove by ensuring they exist in the DataFrame's columns
columns_to_remove_1 = [f"pose_{c}{i}" for c in ['x', 'y', 'z', 'v'] for i in range(0, 11)]
columns_to_remove_2 = [f"pose_{c}{i}" for c in ['x', 'y', 'z', 'v'] for i in range(23, 33)]
columns_to_remove = [col for col in (columns_to_remove_1 + columns_to_remove_2) if col in data.columns]

# Drop these columns from the DataFrame
data_filtered = data.drop(columns=columns_to_remove)

# Save the updated DataFrame to a new CSV file
data_filtered.to_csv('filtered_coordinates.csv', index=False)

['class', 'accuracy', 'sequence', 'pose_x0', 'pose_y0', 'pose_z0', 'pose_v0', 'pose_x1', 'pose_y1', 'pose_z1', 'pose_v1', 'pose_x2', 'pose_y2', 'pose_z2', 'pose_v2', 'pose_x3', 'pose_y3', 'pose_z3', 'pose_v3', 'pose_x4', 'pose_y4', 'pose_z4', 'pose_v4', 'pose_x5', 'pose_y5', 'pose_z5', 'pose_v5', 'pose_x6', 'pose_y6', 'pose_z6', 'pose_v6', 'pose_x7', 'pose_y7', 'pose_z7', 'pose_v7', 'pose_x8', 'pose_y8', 'pose_z8', 'pose_v8', 'pose_x9', 'pose_y9', 'pose_z9', 'pose_v9', 'pose_x10', 'pose_y10', 'pose_z10', 'pose_v10', 'pose_x11', 'pose_y11', 'pose_z11', 'pose_v11', 'pose_x12', 'pose_y12', 'pose_z12', 'pose_v12', 'pose_x13', 'pose_y13', 'pose_z13', 'pose_v13', 'pose_x14', 'pose_y14', 'pose_z14', 'pose_v14', 'pose_x15', 'pose_y15', 'pose_z15', 'pose_v15', 'pose_x16', 'pose_y16', 'pose_z16', 'pose_v16', 'pose_x17', 'pose_y17', 'pose_z17', 'pose_v17', 'pose_x18', 'pose_y18', 'pose_z18', 'pose_v18', 'pose_x19', 'pose_y19', 'pose_z19', 'pose_v19', 'pose_x20', 'pose_y20', 'pose_z20', 'pose_v20'

In [15]:
# Cell 10: Reduce decimal points
data = pd.read_csv('filtered_coordinates.csv')
data = data.round(3)
data.head()

,class,accuracy,sequence,pose_x11,pose_y11,pose_z11,pose_v11,pose_x12,pose_y12,pose_z12,...,left_hand_z18,left_hand_v18,left_hand_x19,left_hand_y19,left_hand_z19,left_hand_v19,left_hand_x20,left_hand_y20,left_hand_z20,left_hand_v20
0,movement_1,0,0,0.378,0.412,-0.578,1,0.151,0.546,-0.492,...,0,0,0,0,0,0,0,0,0,0
1,movement_1,0,0,0.378,0.406,-0.603,1,0.159,0.526,-0.521,...,0,0,0,0,0,0,0,0,0,0
2,movement_1,0,0,0.391,0.416,-0.578,1,0.158,0.570,-0.540,...,0,0,0,0,0,0,0,0,0,0
3,movement_1,0,0,0.381,0.395,-0.452,1,0.158,0.540,-0.390,...,0,0,0,0,0,0,0,0,0,0
4,movement_1,0,0,0.379,0.393,-0.477,1,0.160,0.544,-0.434,...,0,0,0,0,0,0,0,0,0,0
